<a href="https://colab.research.google.com/github/AdiJainS/Decible-Duel/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!docker run --gpus=all -p 127.0.0.1:9000:8080 asia-docker.pkg.dev/colab-images/public/runtime

/bin/bash: line 1: docker: command not found


In [2]:
!pip install torch torchaudio torchvision transformers


In [3]:

# ==============================================================================
# 0. IMPORTS & INITIAL SETUP
# ==============================================================================
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import Audio, display

In [4]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After importing the libraries , first it is important to create an Audio dataset class , where initially we are creating a constroctor called init and gave it some paramets life max_frames , fraction , samplerate etc.. and called them afterwards . We index mapped the class
 --- self.class_to_idx = {cat: i for i, cat in enumerate(categories)} --- > convert string to no.s

 After that we ar looping over the categories and then collects all the audio files.

 Now when we are sampling the files , here the fracntion sample is useful ,
  num_to_sample = int(len(files_in_cat) * fraction)  ---> fraction is set as 1 , hence all the files will be selected , if eg 0.7 was set as frac, randomly any 70% of files would have been selected ,which obviously helps when we do not need to load all the files , and a list of loaded file is then created.

  def __len__(self):
        return len(self.valid_files)  --- > is req to know that how many items are ready to be iterated
      
_getitem_ helps in accessing the files ---- > dunder method

signal -- > tensor of shape [channel , time ] - ie [1,16000]
1 --- > mono 2 ---> stereo  16000 --- > i sec if 32000----> 2 sec.
After that we neeed to make 16k Hz uniform in every audio input , hence we need to resample it . If needed we also can convery stereo to mono.
Now this is pretty important . We need to create spectograms .
n_fft --> we need to analyze chunks of 1024 samples
sr --- > 16000 ie 1024/16000 = 0.064s
hop length -- > controls how far a window can move . 256 means each frame starts with 256 samples after a new one.
n_mels -- >  mel bands compress to spectrum ( ie human perceived freq)

Now we need to make spect. uniform , hence we can eirhter pad or trim them

            _, _, n_frames = log_spec.shape
            if n_frames < self.max_frames:
                pad = self.max_frames - n_frames
                # Pad along the time dimension (last dimension)
                log_spec = F.pad(log_spec, (0, pad))


label_vec = F.one_hot(torch.tensor(label), num_classes=len(self.categories)).float()   ----> OH labeling categories









In [12]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

class AudioDataset(Dataset):
    def __init__(self, root_dir, categories, max_frames=512, fraction=1.0, sr=22050):
        self.root_dir = root_dir
        self.categories = categories
        self.max_frames = max_frames
        self.sr = sr
        self.file_list = []
        self.class_to_idx = {cat: i for i, cat in enumerate(categories)}

        print(f"AudioDataset initialized with root_dir: {self.root_dir}")
        print(f"Categories expected: {self.categories}")


        for cat_name in self.categories:
            cat_dir = os.path.join(root_dir, cat_name)
            print(f"Checking category directory: {cat_dir}")
            files_in_cat = [os.path.join(cat_dir, f) for f in os.listdir(cat_dir) if f.endswith(('.wav', '.mp3'))]
            print(f"Found {len(files_in_cat)} files in {cat_dir} before sampling.")


            num_to_sample = int(len(files_in_cat) * fraction)
            if num_to_sample == 0 and len(files_in_cat) > 0: # Ensure at least one file if available
                 num_to_sample = 1
            sampled_files = random.sample(files_in_cat, num_to_sample)
            print(f"Sampled {len(sampled_files)} files from {cat_dir}")
            label_idx = self.class_to_idx[cat_name]
            self.file_list.extend([(file_path, label_idx) for file_path in sampled_files])

        print(f"Total files found before validation: {len(self.file_list)}")

        # Filter out problematic files during initialization
        self.valid_files = []
        for path, label in self.file_list:
            try:
                # Check if torchaudio can load the file without errors by loading a small part
                torchaudio.load(path, frame_offset=0, num_frames=1)
                self.valid_files.append((path, label))
            except Exception as e:
                print(f"Skipping problematic file {path} during initialization: {e}")
                continue
        print(f"Total valid files after validation: {len(self.valid_files)}")


    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        path, label = self.valid_files[idx] # Use pre-filtered valid_files

        try:
            signal, original_sr = torchaudio.load(path)

            # Resample if necessary
            if original_sr != self.sr:
                resampler = torchaudio.transforms.Resample(original_sr, self.sr)
                signal = resampler(signal)

            # Convert stereo to mono if needed
            if signal.size(0) > 1:
                signal = signal.mean(dim=0, keepdim=True)

            # Compute Mel spectrogram
            mel_spec = torchaudio.transforms.MelSpectrogram(
                sample_rate=self.sr, n_fft=1024, hop_length=256, n_mels=128
            )(signal)
            log_spec = torch.log1p(mel_spec)

            # Pad or trim to max_frames
            _, _, n_frames = log_spec.shape
            if n_frames < self.max_frames:
                pad = self.max_frames - n_frames
                # Pad along the time dimension (last dimension)
                log_spec = F.pad(log_spec, (0, pad))
            else:
                # Trim along the time dimension (last dimension)
                log_spec = log_spec[:, :, :self.max_frames]

            # One-hot encode label
            label_vec = F.one_hot(torch.tensor(label), num_classes=len(self.categories)).float()

            return log_spec, label_vec

        except Exception as e:
            print(f"Error loading or processing file {path}: {e}")

            return None, None

In [6]:
import os

audio_train_path = "/content/drive/MyDrive/the-frequency-quest/train"
audio_files_found = False

for root, dirs, files in os.walk(audio_train_path):
    for file in files:
        if file.endswith(('.wav', '.mp3')):
            print(os.path.join(root, file))
            audio_files_found = True

/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108638-9-0-6.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108638-9-0-2.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/107653-9-0-7.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108357-9-0-27.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/110868-9-0-15.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108638-9-0-5.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108041-9-0-2.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/105425-9-0-14.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/107653-9-0-12.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/101848-9-0-9.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street_music/108041-9-0-6.wav
/content/drive/MyDrive/the-frequency-quest/train/train/street

# CREATING GENERATOR CLASS

Generator inherits nn.module --- takes noise

self.fc = nn.Linear(latent_dim + num_categories, 256 * 8 * 32)

noise vector of latent_dim  , OH encoded num_categroy . latent+category = single vector.
This vector is projected (via Linear) into a large tensor that can be reshaped into a small “image-like” feature map of shape (256, 8, 32).

nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # -> 16x64

            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # -> 32x128
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1), # -> 64x256
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1), # -> 128x512
            nn.ReLU() # Use ReLU to match the log1p output range [0, inf)

----- > this is a deconvolutional laters /upsampling layers which increase with each step from (8×32) up to (128×512),ConvTranspose2d doubles the height and width .
BatchNorm2d ---- >Uses CNN to stabilize training.When generator applies transposed convoltion , the val inside feature map can vary , batch norm feature normalizes them , mean =0 variance = 1.
ReLu ---> introduces non linearity and ensures positive outputs.

def forward .....
----> z+y concatenation

h = self.fc(h)

h = h.view(-1, 256, 8, 32)   ---> turns flat vec into 3D
And the fake spec will be transf to discriminator for training




In [7]:
class CGAN_Generator(nn.Module):
    """ The Forger/Artist """
    def __init__(self, latent_dim, num_categories, spec_shape=(128, 512)):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_categories = num_categories
        self.spec_shape = spec_shape

        # Upsampling architecture
        self.fc = nn.Linear(latent_dim + num_categories, 256 * 8 * 32)
        self.unflatten_shape = (256, 8, 32) # (channels, H, W)

        self.net = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # -> 16x64
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # -> 32x128
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1), # -> 64x256
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1), # -> 128x512
            nn.ReLU() # Use ReLU to match the log1p output range [0, inf)
        )

    def forward(self, z, y):
        # Concatenate noise vector (z) and label (y)
        h = torch.cat([z, y], dim=1)
        h = self.fc(h)
        h = h.view(-1, *self.unflatten_shape)
        fake_spec = self.net(h)
        return fake_spec

# CREATING DISCRIMINATOR CLASS

We need to properly differentiate what Generator and Discriminator are doing to understand what is going on with the code.

Generator creates fake spectograms from noise and labels , whereas discriminator determines it.

Generator spect shows (1, 128, 512) , discri outputs A single scalar/logit per sample (real = 1, fake = 0) ,

generator upsamples (convtrasnpose2d) , whereas discr downsamples (conv2d) ----> most imp

generator uses ReLu(+ve) , discr uses LeakyReLu (smol slope of -ve )



In [8]:
class CGAN_Discriminator(nn.Module):
    """ The Detective/Critic """
    def __init__(self, num_categories, spec_shape=(128, 512)):
        super().__init__()
        self.num_categories = num_categories
        self.spec_shape = spec_shape
        H, W = spec_shape

        # Embedding for the label to match the image dimensions
        self.label_embedding = nn.Linear(num_categories, H * W)

        # Downsampling architecture
        self.net = nn.Sequential(
            # Input channel is 2: 1 for spectrogram, 1 for label map
            nn.Conv2d(2, 32, kernel_size=4, stride=2, padding=1), # -> 64x256
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1), # -> 32x128
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # -> 16x64
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # -> 8x32
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # Final output layer to produce a single logit
            nn.Conv2d(256, 1, kernel_size=(8, 32), stride=1, padding=0) # -> 1x1
        )

    def forward(self, spec, y):
        # Create a channel for the label and concatenate it with the spectrogram
        label_map = self.label_embedding(y).view(-1, 1, *self.spec_shape)
        h = torch.cat([spec, label_map], dim=1)

        # Pass through the network
        logit = self.net(h)
        return logit.view(-1, 1) # Flatten to a single value per item in batch

# # UTILITY FUNCTIONS (GENERATION, SAVING)

More the sample rate , better the quality of catching harmonics , but lesser files computes
z --> numsamples , latemt dim



In [9]:
def generate_audio_gan(generator, category_idx, num_samples, device, sample_rate=22050):
    generator.eval()
    num_categories = generator.num_categories
    latent_dim = generator.latent_dim

    # Prepare label and noise
    y = F.one_hot(torch.tensor([category_idx]), num_classes=num_categories).float().to(device)
    z = torch.randn(num_samples, latent_dim, device=device)

    with torch.no_grad():
        log_spec_gen = generator(z, y)

    # Convert spectrogram back to audio (same as VAE)
    spec_gen = torch.expm1(log_spec_gen)
    spec_gen = spec_gen.squeeze(1)

    inverse_mel = torchaudio.transforms.InverseMelScale(
        n_stft=1024 // 2 + 1, n_mels=128, sample_rate=sample_rate
    ).to(device)
    linear_spec = inverse_mel(spec_gen)

    griffin = torchaudio.transforms.GriffinLim(
        n_fft=1024, hop_length=256, win_length=1024, n_iter=32
    ).to(device)

    waveform = griffin(linear_spec)
    return waveform.cpu()

def save_and_play(wav, sample_rate, filename):
    if wav.dim() > 2: wav = wav.squeeze(0)
    torchaudio.save(filename, wav, sample_rate=sample_rate)
    print(f"Saved to {filename}")
    display(Audio(data=wav.numpy(), rate=sample_rate))

# 4. GAN TRAINING FUNCTION

We have one optimizer to each of the setups  G/D .
Beta 1 = 0.5 Beta 2 = 0.999

B1 = CONTROLS HOW MUCH MOMENTUM IS KEPT FOR GRADIENTS. Higher the B1 , more the smoothness , slower the rkn to rapid changes
B2 = MOMENTUM FOR SQUARED GRADIENTS .

BCEwithlogits -- Binary Cross-Entropy + logits (discriminator output is NOT passed through sigmoid manually).

tqdm -- > display progress bar of each epoch


  
    z = torch.randn(batch_size, latent_dim, device=device)
    fake_specs = generator(z, labels)
We need to generate fake noise samples as well

.detach()  ---> helps not being updated in the same step for discr . But when training generator , we will not wirte .detach() , as we want to pass fake data WITHOUT .detach() - gradients flow to generator.  

Then we need to update generator weights and track the losses.

We will then plot spectograms and egenrate sample audio (specto --> audio)

In [10]:
def train_gan(generator, discriminator, dataloader, device, categories, epochs, lr, latent_dim):
    # Optimizers for each model
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    # Loss function
    criterion = nn.BCEWithLogitsLoss()

    # Create directories for output
    os.makedirs("gan_generated_audio", exist_ok=True)
    os.makedirs("gan_spectrogram_plots", exist_ok=True)

    # Lists to store epoch losses for plotting
    g_losses = []
    d_losses = []

    for epoch in range(1, epochs + 1):
        loop = tqdm(dataloader, desc=f"Epoch {epoch}/{epochs}", leave=True)
        total_d_loss = 0.0
        total_g_loss = 0.0
        num_batches = 0

        for real_specs, labels in loop:
            real_specs = real_specs.to(device)
            labels = labels.to(device)
            batch_size = real_specs.size(0)

            # Labels for loss calculation
            real_labels_tensor = torch.ones(batch_size, 1, device=device)
            fake_labels_tensor = torch.zeros(batch_size, 1, device=device)

            # ---------------------
            #  Train Discriminator
            # ---------------------
            optimizer_D.zero_grad() # Reset gradients.
            real_output = discriminator(real_specs, labels)
            loss_D_real = criterion(real_output, real_labels_tensor)

            z = torch.randn(batch_size, latent_dim, device=device)
            fake_specs = generator(z, labels)

            fake_output = discriminator(fake_specs.detach(), labels)
            loss_D_fake = criterion(fake_output, fake_labels_tensor)

            loss_D = loss_D_real + loss_D_fake
            loss_D.backward()
            optimizer_D.step()

            # -----------------
            #  Train Generator
            # -----------------
            optimizer_G.zero_grad()
            # Generator wants discriminator to think fakes are real
            output = discriminator(fake_specs, labels) # Corrected: pass the original labels
            loss_G = criterion(output, real_labels_tensor)

            loss_G.backward()
            optimizer_G.step()

            total_d_loss += loss_D.item()
            total_g_loss += loss_G.item()
            num_batches += 1

            loop.set_postfix(loss_D=loss_D.item(), loss_G=loss_G.item())

        # Calculate average epoch losses
        avg_d_loss = total_d_loss / num_batches if num_batches > 0 else 0
        avg_g_loss = total_g_loss / num_batches if num_batches > 0 else 0
        d_losses.append(avg_d_loss)
        g_losses.append(avg_g_loss)


        # --- End of Epoch: Generate and save samples ---
        if epoch % 1 == 0: # Generate samples every epoch
            print(f"\n--- Generating Samples for Epoch {epoch} ---")
            generator.eval()

            # --- PLOTTING CODE ---
            fig, axes = plt.subplots(1, len(categories), figsize=(4 * len(categories), 4))
            if len(categories) == 1: axes = [axes] # Make it iterable

            for cat_idx, cat_name in enumerate(categories):
                y_cond = F.one_hot(torch.tensor([cat_idx]), num_classes=generator.num_categories).float().to(device)
                z_sample = torch.randn(1, generator.latent_dim).to(device)
                with torch.no_grad():
                    spec_gen_log = generator(z_sample, y_cond)

                spec_gen_log_np = spec_gen_log.squeeze().cpu().numpy()
                axes[cat_idx].imshow(spec_gen_log_np, aspect='auto', origin='lower', cmap='viridis')
                axes[cat_idx].set_title(f'{cat_name} (Epoch {epoch})')
                axes[cat_idx].axis('off')

            plt.tight_layout()
            plt.savefig(f'gan_spectrogram_plots/epoch_{epoch:03d}.png')
            plt.show() # Optionally show plots during training
            plt.close(fig) # Close the figure to free up memory
            # --- END OF PLOTTING CODE ---

            # --- Audio generation ---
            for cat_idx, cat_name in enumerate(categories):
                wav = generate_audio_gan(generator, cat_idx, 1, device, sample_rate=16000) # Use dataset sample rate
                fname = f"gan_generated_audio/{cat_name}_ep{epoch}.wav"
                save_and_play(wav, sample_rate=16000, filename=fname)

            generator.train() # Set back to training mode
            print("--- End of Sample Generation ---\n")

    return g_losses, d_losses # Return losses

    torch.backends.cudnn.benchmark = True
CUDNN chooses the best kernels , which is constant in size .not reproducible if sizes vary.
    USE_AMP =True
enables mixed precisions to speed up training

BCE ---> Binary crossentropies on prob  ---> gives numerical stability .
Then adam optimizer was seperately applied on Generaor and discri.

scalar = .... is used to prevent underflow.(ie rounding of a very small number to zero)

    (batch_size ,1)  
---> discriminator can only give 1 pred per sample

While training Discr , we need to code about fake and real losses and while training Genr , we need to create new fake data as fake data used in discr is detached [detach()] and generator and discriminator should by independent of each other as Genr is trying to create fake audio samples an Discr is trying to identify fake and real samples.

    if (epoch + 1) % 1 == 0:
-----> This means that after every epoch is loaded , Audio sample and spectogram img is produced .

    

In [14]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import torch.nn.functional as F
import torchaudio

torch.backends.cudnn.benchmark = True  # Auto-select best kernels
USE_AMP = True  # Mixed precision training

#Training
def train_gan_fast(generator, discriminator, dataloader, device, categories,
                   epochs, lr, latent_dim, sample_rate=16000):

    criterion = nn.BCEWithLogitsLoss()

    # Adam optimizers
    opt_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

    scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)

    g_losses = [] #stores generator data
    d_losses = []

    # Create directories for output
    os.makedirs("gan_generated_audio", exist_ok=True)
    os.makedirs("gan_spectrogram_plots", exist_ok=True)

    for epoch in range(epochs):
        progbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        for real_data, labels in progbar:
            real_data = real_data.to(device)  # labels req for conditioning .
            labels = labels.to(device)

            batch_size = real_data.size(0)

            # REAL and FAKE targets
            real_targets = torch.ones(batch_size, 1, device=device)
            fake_targets = torch.zeros(batch_size, 1, device=device)


            #  Train Discriminator

            with torch.cuda.amp.autocast(enabled=USE_AMP):
                # Real loss
                real_preds = discriminator(real_data, labels)
                d_loss_real = criterion(real_preds, real_targets)

                # Fake loss
                noise = torch.randn(batch_size, latent_dim, device=device)
                fake_data = generator(noise, labels)
                fake_preds = discriminator(fake_data.detach(), labels)
                d_loss_fake = criterion(fake_preds, fake_targets)

                d_loss = d_loss_real + d_loss_fake

            opt_D.zero_grad()
            scaler.scale(d_loss).backward()
            scaler.step(opt_D)


            #  Train Generator

            with torch.cuda.amp.autocast(enabled=USE_AMP):
                noise = torch.randn(batch_size, latent_dim, device=device)
                fake_data = generator(noise, labels)
                preds = discriminator(fake_data, labels)
                g_loss = criterion(preds, real_targets)

            opt_G.zero_grad()
            scaler.scale(g_loss).backward()
            scaler.step(opt_G)
            scaler.update()

            g_losses.append(g_loss.item())
            d_losses.append(d_loss.item())

            progbar.set_postfix({"D": d_loss.item(), "G": g_loss.item()})


        if (epoch + 1) % 1 == 0: # Generate samples every epoch
            print(f"\n--- Generating Samples for Epoch {epoch+1} ---")
            generator.eval()

            # SPECTOGRAM CODE
            fig, axes = plt.subplots(1, len(categories), figsize=(4 * len(categories), 4))
            if len(categories) == 1: axes = [axes] # Make it iterable

            for cat_idx, cat_name in enumerate(categories):
                y_cond = F.one_hot(torch.tensor([cat_idx]), num_classes=generator.num_categories).float().to(device)
                z_sample = torch.randn(1, generator.latent_dim).to(device)
                with torch.no_grad():
                    spec_gen_log = generator(z_sample, y_cond)

                spec_gen_log_np = spec_gen_log.squeeze().cpu().numpy()
                axes[cat_idx].imshow(spec_gen_log_np, aspect='auto', origin='lower', cmap='viridis')
                axes[cat_idx].set_title(f'{cat_name} (Epoch {epoch+1})')
                axes[cat_idx].axis('off')

            plt.tight_layout()
            plt.savefig(f'gan_spectrogram_plots/epoch_{epoch+1:03d}.png')
            plt.show()
            plt.close(fig) # Close the figure to free up memory


            # --- Audio generation ---
            for cat_idx, cat_name in enumerate(categories):
                wav = generate_audio_gan(generator, cat_idx, 1, device, sample_rate=sample_rate)
                fname = f"gan_generated_audio/{cat_name}_ep{epoch+1}.wav"
                save_and_play(wav, sample_rate=sample_rate, filename=fname)

            generator.train() # Set back to training mode
            print("--- End of Sample Generation ---\n")

    return g_losses, d_losses


#Execution
if __name__ == '__main__':

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    LATENT_DIM = 100
    EPOCHS = 100  # reduced for speed
    BATCH_SIZE = 128  # faster
    LEARNING_RATE = 0.0001
    SAMPLE_RATE = 22050 # Set sample rate as used in AudioDataset

    DATA_ROOT_PATH = '/content/drive/MyDrive/the-frequency-quest/train/train'

    train_categories = sorted([
        d for d in os.listdir(DATA_ROOT_PATH)
        if os.path.isdir(os.path.join(DATA_ROOT_PATH, d))
    ])
    NUM_CATEGORIES = len(train_categories)

    print(f"Using device: {DEVICE}")
    print(f"Categories: {train_categories}")

    train_dataset = AudioDataset(
        root_dir=DATA_ROOT_PATH,
        categories=train_categories,
        sr=SAMPLE_RATE
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,      # faster loading
        pin_memory=True     # GPU boost
    )

    generator = CGAN_Generator(LATENT_DIM, NUM_CATEGORIES).to(DEVICE)
    discriminator = CGAN_Discriminator(NUM_CATEGORIES).to(DEVICE)

    g_losses, d_losses = train_gan_fast(
        generator=generator,
        discriminator=discriminator,
        dataloader=train_loader,
        device=DEVICE,
        categories=train_categories,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        latent_dim=LATENT_DIM,
        sample_rate=SAMPLE_RATE
    )

    print("Training complete.")

Using device: cuda
Categories: ['dog_bark', 'drilling', 'engine_idling', 'siren', 'street_music']
AudioDataset initialized with root_dir: /content/drive/MyDrive/the-frequency-quest/train/train
Categories expected: ['dog_bark', 'drilling', 'engine_idling', 'siren', 'street_music']
Checking category directory: /content/drive/MyDrive/the-frequency-quest/train/train/dog_bark
Found 704 files in /content/drive/MyDrive/the-frequency-quest/train/train/dog_bark before sampling.
Sampled 704 files from /content/drive/MyDrive/the-frequency-quest/train/train/dog_bark
Checking category directory: /content/drive/MyDrive/the-frequency-quest/train/train/drilling
Found 700 files in /content/drive/MyDrive/the-frequency-quest/train/train/drilling before sampling.
Sampled 700 files from /content/drive/MyDrive/the-frequency-quest/train/train/drilling
Checking category directory: /content/drive/MyDrive/the-frequency-quest/train/train/engine_idling
Found 700 files in /content/drive/MyDrive/the-frequency-quest